In [ ]:
!pip install yfinance
import pandas as pd
import yfinance as yf # import libraries
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import warnings
warnings.filterwarnings('ignore') 
# from IPython.display import FileLink
from google.colab import files
import tqdm.notebook as tq

     |████████████████████████████████| 6.3MB 6.3MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23919 sha256=4f5fdb84ac6ac7e9d5b7bf7c5e55576518f887fa5d3f9d5dbdf50baca31e039f
  Stored in directory: /root/.cache/pip/wheels/fe/a0/79/b73d4a0c535b421b88fc7b393936b371fabbfeaf979eca4050
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
from pandas_datareader import data as pdr
yf.pdr_override() # <== that's all it takes :-)

# BACKTESTING

In [ ]:
def checker(stock_name,start_date,end_date): #,Amount):
  data = pdr.get_data_yahoo(stock_name, 
                    start=start_date, 
                    end=end_date,
                    progress=False)
  # Create levels
  data['R4']= (data['Close'] + ((data['High']-data['Low'])*(1.1/2))).shift(1)
  data['R3']= (data['Close'] + ((data['High']-data['Low'])*(1.1/4))).shift(1)
  data['S3']= (data['Close'] - ((data['High']-data['Low'])*(1.1/4))).shift(1)
  data['S4']= (data['Close'] - ((data['High']-data['Low'])*(1.1/2))).shift(1)
  # Check if camarilla got activated
  data['Camarilla']=['Yes' if (data['R3'][i-1] > data['R3'][i] and data['S3'][i-1] < data['S3'][i]) else 'No' for i in range(0,len(data)) ]
  data=data[data['Camarilla']== 'Yes']
  # Calculate profit if activated
  data['P/L']=[(data['Close'][i] - data['R4'][i]) if (data['R4'][i] < data['High'][i]) else  (data['S4'][i] - data['Close'][i]) if (data['Low'][i] < data['S4'][i]) else 0 for i in range(0,len(data))]
  data.reset_index(inplace=True)
  data['Date']=data['Date'].astype('str')
  # data['INR'] = [((round(Amount/data['R4'][i])) * data['P/L'][i]) if (data['R4'][i] < data['High'][i]) else ((round(Amount/data['S4'][i])) * data['P/L'][i]) for i in range(0,len(data))]
  # return sum(data['INR']), len(data), list(data['Date']) 
  return len(data), list(data['Date']) 

In [ ]:
# from tqdm import tqdm
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
%cd '/content/gdrive/MyDrive/SASH/'
# nse=pd.read_csv('ALL_FNO.csv')
# nse

/content/gdrive/MyDrive/SASH


In [ ]:
# files.download('ALL_FNO.csv')

In [ ]:
nse=pd.read_csv('/content/gdrive/MyDrive/SASH/Nifty50.csv')
nse['Symbol']= nse['Symbol'] + '.NS'

In [ ]:
checker('CADILAHC.NS','2021-04-16','2021-04-22') #,100000)

(1, ['2021-04-20'])

In [ ]:
# create df of nested info per stock
selected_stocks={}
for i in tq.tqdm(nse['Symbol']):
    selected_stocks[i]=checker(i,'2021-06-01','2021-06-30')
final_selection=pd.DataFrame.from_dict(selected_stocks,orient='index',columns=['Close','Dates'])
final_selection.reset_index(inplace=True)
final_selection=final_selection.rename(columns={'index':'Ticker'})
final_selection=final_selection.reset_index().drop('index',axis=1)
# add all in proper list format
new_col=[]
for i in tq.tqdm(range(0,len(final_selection))):
  new_entry=[]
  for j in range(0,len(final_selection['Dates'][i])):
    new_entry.append([final_selection['Ticker'][i],final_selection['Dates'][i][j]])
  new_col.append(new_entry)
# make it to proper csv file
final = pd.DataFrame(columns=['Stock','Date'])
for i in tq.tqdm(range(0,len(new_col))):
  for j in new_col[i]:
    try:
      final = final.append({'Stock': j[0],'Date':j[1]}, ignore_index=True)
    except:
      pass
print('completed, please check dataframe "final" ')
final=final.dropna().reset_index(drop=True)
final['Date']=final['Date'].astype('str')
final['Stock']='NSE:' + final['Stock'].str.replace('.NS',',')


completed, please check dataframe "final" 


In [ ]:
final.to_excel('cam_50_jun.xlsx',index=False)
files.download('cam_50_jun.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# bull_sash_stocks={}
# for i in tq.tqdm(nse['Symbol']):
#     bull_sash_stocks[i]=checker(i,'2021-03-01','2021-03-30',100000)
# final_bull_sash=pd.DataFrame.from_dict(bull_sash_stocks,orient='index',columns=['P/L','Counts','Dates'],)
# final_bull_sash.reset_index(inplace=True)
# final_bull_sash=final_bull_sash.rename(columns={'index':'Ticker'})
# final_bull_sash=final_bull_sash[final_bull_sash['Counts'] != 0]
# print('\n')
# print('Total Trades:',sum(final_bull_sash['Counts']))
# print('Total INR:',sum(final_bull_sash['P/L']))
# final_bull_sash=final_bull_sash.reset_index().drop('index',axis=1)
# # final_bull_sash.to_csv('Nifty50May.csv',index=False)




Total Trades: 71
Total INR: 20145.191041946415


In [ ]:
# files.download('camarilla_test_Feb.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# TO PREDICT NEXT DAY STOCKS

In [ ]:
# from tqdm import tqdm
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
%cd '/content/gdrive/MyDrive/SASH/'

/content/gdrive/MyDrive/SASH


In [ ]:
nse=pd.read_csv('ALL_FNO.csv')

In [ ]:
def stock_selector(stock_name,start_date,end_date):
  data = yf.download(stock_name, 
                    start=start_date, 
                    end=end_date,
                    progress=False)
# Create levels
  data['R4']= data['Close'] + ((data['High']-data['Low'])*(1.1/2))
  data['R3']= data['Close'] + ((data['High']-data['Low'])*(1.1/4))
  data['S3']= data['Close'] - ((data['High']-data['Low'])*(1.1/4))
  data['S4']= data['Close'] - ((data['High']-data['Low'])*(1.1/2))
# Check if camarilla got activated
  data['Camarilla']=['Yes' if (data['R3'][i-1] > data['R3'][i] and data['S3'][i-1] < data['S3'][i]) else 'No' for i in range(0,len(data)) ]
  data=data[data['Camarilla']== 'Yes']
  data.reset_index(inplace=True)
  data['Date']=data['Date'].astype('str')
# take only last date as we want to predict only for next day
  data=data[-1:]
  if data.empty:
    return list(data['Date']), list(data['R4']), list(data['R3']) , list(data['S4']) , list(data['S3'])
  else:
    df=data['Date'].str.split('-',expand=True)
    return int(df[2]), float(data['R4']),float(data['R3']),float(data['S4']),float(data['S3'])

In [ ]:
# %cd '/content/gdrive/MyDrive/SASH/'
# nse=pd.read_csv('new_FNO_list.csv')

stocks={}
for i in tq.tqdm(nse['Symbol']):
  # provide range
  stocks[i]=stock_selector(i,'2021-07-09','2021-07-16')
# add column names and convert to df
selected_stocks=pd.DataFrame.from_dict(stocks,orient='index',columns=['Date','Buy-Bull','SL-Bull','Sell-Bear','SL-Bear'])
# slice on mentioned date
selected_stocks=selected_stocks[selected_stocks["Date"] == 15]
# reset index and rename 
selected_stocks=selected_stocks.reset_index()
selected_stocks=selected_stocks.rename(columns={'index':'Stock'})
# Drop if buy or sell == stoploss
for i in range(0,len(selected_stocks)):
  if selected_stocks['Buy-Bull'][i] == selected_stocks['SL-Bull'][i] and selected_stocks['Sell-Bear'][i ] == selected_stocks['SL-Bear'][i]:
    selected_stocks = selected_stocks.drop(i)
selected_stocks

,Stock,Date,Buy-Bull,SL-Bull,Sell-Bear,SL-Bear
0,DRREDDY.NS,15,5430.66,5412.16,5356.64,5375.14
1,IGL.NS,15,564.655,562.428,555.745,557.973
2,NMDC.NS,15,173.695,173.172,171.605,172.127
3,PIDILITIND.NS,15,2259.85,2254.5,2238.45,2243.8
4,TATACHEM.NS,15,782.42,780.11,773.18,775.49
5,SBIN.NS,15,433.515,432.608,429.885,430.793
6,SHREECEM.NS,15,28608.2,28517.8,28246.7,28337.1
7,MINDTREE.NS,15,2770.46,2751.28,2693.74,2712.92


In [ ]:
selected_stocks['Buy-Quantity']=100000/selected_stocks['Buy-Bull']
selected_stocks['Buy-Quantity']=selected_stocks['Buy-Quantity'].astype('int')
selected_stocks['Sell-Quantity']=100000/selected_stocks['Sell-Bear']
selected_stocks['Sell-Quantity']=selected_stocks['Sell-Quantity'].astype('int')

In [ ]:
asset=selected_stocks['Stock'].str.split(".",expand=True)
selected_stocks['Stock']= "NSE:" + asset[0] + ","

In [ ]:
selected_stocks

,Stock,Date,Buy-Bull,SL-Bull,Sell-Bear,SL-Bear,Buy-Quantity,Sell-Quantity
0,"NSE:DRREDDY,",15,5430.66,5412.16,5356.64,5375.14,18,18
1,"NSE:IGL,",15,564.655,562.428,555.745,557.973,177,179
2,"NSE:NMDC,",15,173.695,173.172,171.605,172.127,575,582
3,"NSE:PIDILITIND,",15,2259.85,2254.5,2238.45,2243.8,44,44
4,"NSE:TATACHEM,",15,782.42,780.11,773.18,775.49,127,129
5,"NSE:SBIN,",15,433.515,432.608,429.885,430.793,230,232
6,"NSE:SHREECEM,",15,28608.2,28517.8,28246.7,28337.1,3,3
7,"NSE:MINDTREE,",15,2770.46,2751.28,2693.74,2712.92,36,37


In [ ]:
selected_stocks.to_excel('stockselection_16th_july.xlsx',index=False)
files.download('stockselection_16th_july.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
rm 'stockselection_16th_july.xlsx'